# Introduction à SPARK MLIB

Ce notebook est conçu pour vous aider à démarrer avec Apache Spark MLlib.

Nous allons étudier un problème de classification binaire:
**Prédire si le revenu d'un individu est supérieur à 50 000 $** à partir d'une base de données démographiques? 

L'ensemble de données provient du référentiel [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult) et est fourni avec Databricks Runtime. 

Ce bloc-notes présente certaines des fonctionnalités disponibles dans MLlib: notamment des outils de prétraitement des données, des pipelines d'apprentissage automatique et plusieurs algorithmes d'apprentissage automatique.

<b>Ce cahier comprend les étapes suivantes:</b> 

0. Charger le jeu de données
0. Nettoayage des données
0. Définir le modèle
0. Construisez le pipeline
0. Évaluer le modèle
0. Réglage des hyperparamètres
0. Faire des prédictions et évaluer les performances du modèle

# Étape 0. Chargez l'ensemble de données

Afficher les premières lignes des données.

Créer un schéma pour attribuer des noms de colonne et des types de données.

In [0]:
schema = """`age` DOUBLE,
`workclass` STRING,
`fnlwgt` DOUBLE,
`education` STRING,
`education_num` DOUBLE,
`marital_status` STRING,
`occupation` STRING,
`relationship` STRING,
`race` STRING,
`sex` STRING,
`capital_gain` DOUBLE,
`capital_loss` DOUBLE,
`hours_per_week` DOUBLE,
`native_country` STRING,
`income` STRING"""

dataset = spark.read.csv("/databricks-datasets/adult/adult.data", schema=schema)

#dataset = spark.read.csv("/databricks-datasets/adult/adult.data", header=True, inferSchema=True)
dataset.show()
print(dataset.count())

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
 age| workclass| fnlwgt| education|education_num| marital_status| occupation| relationship| race| sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
39.0| State-gov| 77516.0| Bachelors| 13.0| Never-married| Adm-clerical| Not-in-family| White| Male| 2174.0| 0.0| 40.0| United-States| <=50K|
50.0| Self-emp-not-inc| 83311.0| Bachelors| 13.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 0.0| 0.0| 13.0| United-States| <=50K|
38.0| Private|215646.0| HS-grad| 9.0| Divorced| Handlers-cleaners| Not-in-family| White| Male| 0.0| 0.0| 40.0| United-States| <=50K|
53.0| Private|234721.0| 11th| 7.0| Married-civ-spouse| Handlers-cleaners| Husband| Black| Male| 0.0| 0.0| 40.0| United-States| <=50K|
28.0| Private|338409.0| Bachelors| 13.0| Married-civ-spouse| Prof-specialty| Wife| Black| Female| 0.0| 0.0| 40.0| Cuba| <=50K|
37.0| Private|284582.0| Masters| 14.0| Married-civ-spouse| Exec-managerial| Wife| White| Female| 0.0| 0.0| 40.0| United-States| <=50K|
49.0| Private|160187.0| 9th| 5.0| Married-spouse-a...| Other-service| Not-in-family| Black| Female| 0.0| 0.0| 16.0| Jamaica| <=50K|
52.0| Self-emp-not-inc|209642.0| HS-grad| 9.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 0.0| 0.0| 45.0| United-States| >50K|
31.0| Private| 45781.0| Masters| 14.0| Never-married| Prof-specialty| Not-in-family| White| Female| 14084.0| 0.0| 50.0| United-States| >50K|
42.0| Private|159449.0| Bachelors| 13.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 5178.0| 0.0| 40.0| United-States| >50K|
37.0| Private|280464.0| Some-college| 10.0| Married-civ-spouse| Exec-managerial| Husband| Black| Male| 0.0| 0.0| 80.0| United-States| >50K|
30.0| State-gov|141297.0| Bachelors| 13.0| Married-civ-spouse| Prof-specialty| Husband| Asian-Pac-Islander| Male| 0.0| 0.0| 40.0| India| >50K|
23.0| Private|122272.0| Bachelors| 13.0| Never-married| Adm-clerical| Own-child| White| Female| 0.0| 0.0| 30.0| United-States| <=50K|
32.0| Private|205019.0| Assoc-acdm| 12.0| Never-married| Sales| Not-in-family| Black| Male| 0.0| 0.0| 50.0| United-States| <=50K|
40.0| Private|121772.0| Assoc-voc| 11.0| Married-civ-spouse| Craft-repair| Husband| Asian-Pac-Islander| Male| 0.0| 0.0| 40.0| ?| >50K|
34.0| Private|245487.0| 7th-8th| 4.0| Married-civ-spouse| Transport-moving| Husband| Amer-Indian-Eskimo| Male| 0.0| 0.0| 45.0| Mexico| <=50K|
25.0| Self-emp-not-inc|176756.0| HS-grad| 9.0| Never-married| Farming-fishing| Own-child| White| Male| 0.0| 0.0| 35.0| United-States| <=50K|
32.0| Private|186824.0| HS-grad| 9.0| Never-married| Machine-op-inspct| Unmarried| White| Male| 0.0| 0.0| 40.0| United-States| <=50K|
38.0| Private| 28887.0| 11th| 7.0| Married-civ-spouse| Sales| Husband| White| Male| 0.0| 0.0| 50.0| United-States| <=50K|
43.0| Self-emp-not-inc|292175.0| Masters| 14.0| Divorced| Exec-managerial| Unmarried| White| Female| 0.0| 0.0| 45.0| United-States| >50K|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
only showing top 20 rows

32561

Divisez aléatoirement les données en ensembles d'apprentissage et de test.

In [0]:
trainDF, testDF = dataset.randomSplit([0.8, 0.2], seed=42)
print(trainDF.cache().count()) # Cache car accès aux données d'entraînement plusieurs fois
print(testDF.count())

26076
6485

# Visualiser les données

In [0]:
display(trainDF.select("hours_per_week").summary())

summary,hours_per_week
count,26076
mean,40.4284782942169
stddev,12.404569739132008
min,1.0
25%,40.0
50%,40.0
75%,45.0
max,99.0


In [0]:
display(trainDF.filter(trainDF.education_num == 13.0) \
    .sort("age") \
    .groupBy("marital_status")\
    .count())
    

marital_status,count
Widowed,66
Married-spouse-absent,56
Married-AF-spouse,4
Married-civ-spouse,2178
Divorced,430
Never-married,1445
Separated,76


In [0]:
display(trainDF.filter(trainDF.education_num == 13.0) \
    .groupBy("age")\
    .count()\
    .sort("age"))

age,count
19.0,2
20.0,1
21.0,4
22.0,76
23.0,144
24.0,154
25.0,169
26.0,167
27.0,154
28.0,151


In [0]:
display(trainDF
        .groupBy("education")
        .count()
        .sort("count", ascending=False))

education,count
HS-grad,8408
Some-college,5860
Bachelors,4255
Masters,1388
Assoc-voc,1102
11th,958
Assoc-acdm,845
10th,748
7th-8th,510
Prof-school,465


In [0]:
display(trainDF
        .groupBy("income")
        .count()
        .sort("count", ascending=False))

income,count
<=50K,19812
>50K,6264


# Transformers, estimators, and pipelines

-**Transformer:** prend un DataFrame comme entrée et renvoie un nouveau DataFrame. Les transformateurs n'apprennent aucun paramètre à partir des données. Ils appliquent simplement des transformations basées sur des règles pour **préparer les données** pour l'apprentissage du modèle **ou générer des prédictions à l'aide d'un modèle MLlib entraîné **. On appele un transformateur avec une méthode '.transform ()'.

-**Estimator:** **apprend** (ou «ajuste») les paramètres de votre DataFrame via une méthode '.fit ()' et renvoie un modèle, qui est un transformateur.

-**Pipeline:** combine plusieurs étapes dans un flux de travail unique qui peut être facilement exécuté. La création d'un modèle d'apprentissage automatique implique généralement la configuration de nombreuses étapes différentes et leur itération. Les pipelines vous aident à automatiser ce processus.

**Comment ça fonctionne?**
Un pipeline est spécifié comme une séquence d'étapes et **chaque étape est soit un transformateur, soit un estimateur**. Ces étapes sont exécutées dans l'ordre et le DataFrame d'entrée est transformé à mesure qu'il traverse chaque étape. Pour les étapes Transformer, la méthode transform () est appelée sur le DataFrame. Pour les étapes Estimator, la méthode fit () est appelée pour produire un Transformer (qui devient une partie du PipelineModel, ou Pipeline ajusté), et la méthode transform () de Transformer est appelée sur le DataFrame.

In [0]:
Image(url= "https://spark.apache.org/docs/1.6.0/img/ml-PipelineModel.png")

Out[21]:

Au-dessus, la rangée du haut représente un pipeline à trois étapes. Les deux premiers (Tokenizer et HashingTF) sont des Transformers ( et le troisième (LogisticRegression) est un Estimator. La ligne du bas représente les données circulant dans le pipeline, où les cylindres indiquent les DataFrames. La méthode Pipeline.fit () est appelée sur le DataFrame d'origine, qui contient des documents texte bruts et des étiquettes. La méthode Tokenizer.transform () divise les documents texte brut en mots, ajoutant une nouvelle colonne avec des mots au DataFrame. La méthode HashingTF.transform () convertit la colonne de mots en vecteurs de caractéristiques, en ajoutant une nouvelle colonne avec ces vecteurs au DataFrame. Maintenant, puisque LogisticRegression est un Estimator, le Pipeline appelle d'abord LogisticRegression.fit () pour produire un LogisticRegressionModel. Si le pipeline avait plus d'étapes, il appellerait la méthode transform () de LogisticRegressionModel sur le DataFrame avant de passer le DataFrame à l'étape suivante.

Un pipeline est un estimateur. Ainsi, après l’exécution de la méthode fit () d’un Pipeline, elle produit un PipelineModel, qui est un Transformer. Ce PipelineModel est utilisé au moment du test; la figure ci-dessous illustre cette utilisation.

Exemple de Spark ML PipelineModel

Dans la figure ci-dessus, le PipelineModel a le même nombre d'étapes que le Pipeline d'origine, mais tous les Estimateurs du Pipeline d'origine sont devenus des Transformers. Lorsque la méthode transform () de PipelineModel est appelée sur un ensemble de données de test, les données sont transmises dans le pipeline ajusté dans l'ordre. La méthode transform () de chaque étape met à jour l'ensemble de données et le transmet à l'étape suivante.

Les pipelines et les PipelineModels permettent de garantir que les données d'entraînement et de test passent par des étapes de traitement des fonctionnalités identiques.

[ML Pipelines](https://spark.apache.org/docs/latest/ml-pipeline.html#ml-pipelines)

# Étape 1. Nettoayage des données
On veut construire un modèle qui prédit le niveau de revenu à partir des caractéristiques incluses dans l'ensemble de données (niveau d'éducation, état matrimonial, profession, etc.). La première étape consiste à manipuler ou prétraiter les données afin qu'elles soient au format requis par MLlib.

## Convertir les variables catégorielles en variables numériques

Certains algorithmes d'apprentissage automatique, tels que la régression linéaire et logistique, nécessitent des fonctionnalités numériques. L'ensemble de données Adultes comprend des caractéristiques catégoriques telles que l'éducation, la profession et l'état matrimonial.

Le bloc de code suivant illustre comment utiliser `StringIndexer` et` OneHotEncoder` pour convertir des variables catégorielles en un ensemble de variables numériques qui ne prennent que les valeurs 0 et 1.

- `StringIndexer` convertit une colonne de valeurs strings en une colonne d'index d'étiquette. Par exemple, il peut convertir les valeurs «rouge», «bleu» et «vert» en 0, 1 et 2.
- `OneHotEncoder` mappe une colonne d'indices de catégorie à une colonne de vecteurs binaires, avec au plus un" 1 "dans chaque ligne qui indique l'index de catégorie pour cette ligne.

L'encodage One-hot dans Spark est un processus en deux étapes. Vous utilisez d'abord le StringIndexer, suivi de OneHotEncoder. Le bloc de code suivant définit le StringIndexer et OneHotEncoder mais ne l'applique pas encore à des données.

[StringIndexer](http://spark.apache.org/docs/latest/ml-features.html#stringindexer)   
[OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categoricalCols = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex"]

# The following two lines are estimators. They return functions that we will later apply to transform the dataset.
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols]) 
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols]) 

# The label column ("income") is also a string value - it has two possible values, "<=50K" and ">50K". 
# Convert it to a numeric value using StringIndexer.
labelToIndex = StringIndexer(inputCol="income", outputCol="label")

Nous allons créer un pipeline combinant toutes nos étapes de nettoayage et apprentisage. Mais prenons une minute pour examiner de plus près le fonctionnement des estimateurs et des transformateurs en appliquant l'estimateur `stringIndexer` que nous avons créé dans le bloc de code précédent.

Vous pouvez appeler la méthode `.fit ()` pour renvoyer un `StringIndexerModel`, que vous pouvez ensuite utiliser pour transformer l'ensemble de données.

La méthode `.transform ()` de `StringIndexerModel` renvoie un nouveau DataFrame avec les nouvelles colonnes ajoutées. Faites défiler vers la droite pour voir les nouvelles colonnes si nécessaire.

Pour plus d'informations: [StringIndexerModel]

In [0]:
stringIndexerModel = stringIndexer.fit(trainDF)
display(stringIndexerModel.transform(trainDF))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,educationIndex,raceIndex,occupationIndex,relationshipIndex,workclassIndex,marital_statusIndex,sexIndex
17.0,?,34019.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,34088.0,12th,8.0,Never-married,?,Own-child,White,Female,0.0,0.0,25.0,United-States,<=50K,11.0,0.0,7.0,2.0,3.0,1.0,1.0
17.0,?,47407.0,11th,7.0,Never-married,?,Own-child,White,Male,0.0,0.0,10.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,48703.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,1.0
17.0,?,48751.0,11th,7.0,Never-married,?,Own-child,Black,Female,0.0,0.0,40.0,United-States,<=50K,5.0,1.0,7.0,2.0,3.0,1.0,1.0
17.0,?,67808.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,86786.0,10th,6.0,Never-married,?,Own-child,White,Female,0.0,0.0,40.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,1.0
17.0,?,89870.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,94366.0,10th,6.0,Never-married,?,Other-relative,White,Male,0.0,0.0,6.0,United-States,<=50K,7.0,0.0,7.0,5.0,3.0,1.0,0.0
17.0,?,103810.0,12th,8.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K,11.0,0.0,7.0,2.0,3.0,1.0,0.0


### Combine toutes les colonnes d'entités en un seul vecteur d'entités

La plupart des algorithmes MLlib nécessitent une seule colonne de fonctionnalités en entrée. Chaque ligne de cette colonne contient un vecteur  de données correspondant à l'ensemble des caractéristiques utilisées pour la prédiction.

MLlib fournit le transformateur `VectorAssembler` pour créer une seule colonne vectorielle à partir d'une liste de colonnes.

Le bloc de code suivant montre comment utiliser VectorAssembler.

Pour plus d'informations: [VectorAssembler] (https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)

In [0]:
from pyspark.ml.feature import VectorAssembler

# This includes both the numeric columns and the one-hot encoded binary vector columns in our dataset.
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "OHE" for c in categoricalCols] + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

## Étape 2. Définir le modèle d'apprentisage

On utilise un modèle de [régression logistique](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression).

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label", regParam=1.0)

## Étape 3. Construisez le pipeline

Un «Pipeline» est une liste ordonnée de transformateurs et d'estimateurs. Vous pouvez définir un pipeline pour automatiser et garantir la répétabilité des transformations à appliquer à un ensemble de données. Dans cette étape, nous définissons le pipeline, puis nous l'appliquons à l'ensemble de données de test.

Semblable à ce que nous avons vu avec `StringIndexer`, un` Pipeline` est un estimateur. La méthode `pipeline.fit ()` renvoie un `PipelineModel`, qui est un transformateur.

Pour plus d'informations:
[Pipeline] (https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Pipeline)
[PipelineModel] (https://spark.apache.org/docs/latest/api/java/org/apache/spark/ml/PipelineModel.html)

In [0]:
from pyspark.ml import Pipeline

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[stringIndexer, encoder, labelToIndex, vecAssembler, lr])

# Define the pipeline model.
pipelineModel = pipeline.fit(trainDF)

# Apply the pipeline model to the test dataset.
predDF = pipelineModel.transform(testDF)
display(predDF)

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,educationIndex,raceIndex,occupationIndex,relationshipIndex,workclassIndex,marital_statusIndex,sexIndex,relationshipOHE,sexOHE,raceOHE,marital_statusOHE,educationOHE,workclassOHE,occupationOHE,label,features,rawPrediction,probability,prediction
17.0,?,41643.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,15.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,1.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 6, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 14, indices -> List(7), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 41643.0, 7.0, 15.0))","Map(vectorType -> dense, length -> 2, values -> List(2.268654088647896, -2.268654088647896))","Map(vectorType -> dense, length -> 2, values -> List(0.9062474976435643, 0.09375250235643576))",0.0
17.0,?,64785.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,30.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 6, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(7), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 14, indices -> List(7), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 59, indices -> List(3, 15, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 64785.0, 6.0, 30.0))","Map(vectorType -> dense, length -> 2, values -> List(2.119343829172607, -2.119343829172607))","Map(vectorType -> dense, length -> 2, values -> List(0.8927691288853388, 0.10723087111466127))",0.0
17.0,?,80077.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,20.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,1.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 6, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 14, indices -> List(7), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 80077.0, 7.0, 20.0))","Map(vectorType -> dense, length -> 2, values -> List(2.2437461113192247, -2.2437461113192247))","Map(vectorType -> dense, length -> 2, values -> List(0.9041097206748728, 0.09589027932512711))",0.0
17.0,?,104025.0,11th,7.0,Never-married,?,Own-child,White,Male,0.0,0.0,18.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,0.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> s

Display the predictions from the model. The features column is a sparse vector, which is often the case after one-hot encoding, because there are so many 0 values.

In [0]:
display(predDF.select("features", "label", "prediction", "probability"))

features,label,prediction,probability
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 41643.0, 7.0, 15.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9062474976435643, 0.09375250235643576))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 15, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 64785.0, 6.0, 30.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8927691288853388, 0.10723087111466127))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 80077.0, 7.0, 20.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9041097206748728, 0.09589027932512711))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 104025.0, 7.0, 18.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8952738661074835, 0.10472613389251656))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 15, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 139183.0, 6.0, 15.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9087696046250343, 0.09123039537496568))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 8, 24, 36, 45, 49, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 148769.0, 9.0, 40.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8861377407390781, 0.11386225926092175))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 170320.0, 7.0, 8.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9089516898879646, 0.09104831011203536))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 198797.0, 7.0, 20.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8942281267283682, 0.10577187327163177))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 19, 24, 36, 45, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 241021.0, 8.0, 40.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8945663579567216, 0.10543364204327836))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 49, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 250541.0, 7.0, 8.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9099365057082272, 0.09006349429177292))"


## Étape 4. Évaluer le modèle

La commande `display` a une option de courbe ROC intégrée.

To evaluate the model, we use the `BinaryClassificationEvaluator` to evalute the area under the ROC curve and the `MulticlassClassificationEvaluator` to evalute the accuracy.

For more information:  
[BinaryClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator)  
[MulticlassClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator)

In [0]:
display(pipelineModel.stages[-1], predDF.drop("prediction", "rawPrediction", "probability"), "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.5139985144933676
0.0,0.038461538461538464,0.5139985144933676
0.0,0.07692307692307693,0.509717350710216
0.0,0.11538461538461539,0.5001046759516454
0.0,0.15384615384615385,0.4996941428384745
0.010526315789473684,0.15384615384615385,0.4974508751483025
0.010526315789473684,0.19230769230769232,0.4908963582806486
0.010526315789473684,0.23076923076923078,0.462090468283529
0.010526315789473684,0.2692307692307692,0.4355582066136955
0.010526315789473684,0.3076923076923077,0.43179699067681854


In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

bcEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print(f"Area under ROC curve: {bcEvaluator.evaluate(predDF)}")

mcEvaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"Accuracy: {mcEvaluator.evaluate(predDF)}")

Area under ROC curve: 0.8832451799218509
Accuracy: 0.7680801850424056

## Étape 5. Réglage des hyperparamètres

MLlib fournit des méthodes pour faciliter le réglage des hyperparamètres et la validation croisée.
- Pour le réglage des hyperparamètres, `ParamGridBuilder` vous permet de définir une recherche de grille sur un ensemble d'hyperparamètres de modèle.
- Pour la validation croisée, `CrossValidator` vous permet de spécifier un estimateur (le pipeline à appliquer à l'ensemble de données d'entrée), un évaluateur, un espace de grille d'hyperparamètres et le nombre de plis à utiliser pour la validation croisée.
  
Pour plus d'informations:
[Sélection du modèle par validation croisée] (https://spark.apache.org/docs/latest/ml-tuning.html)
[ParamGridBuilder] (https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.tuning)
[CrossValidator] (https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator)

Utilisez `ParamGridBuilder` et` CrossValidator` pour régler le modèle. Cet exemple utilise trois valeurs pour `regParam` et trois pour` elasticNetParam`, pour un total de 3 x 3 = 9 combinaisons d'hyperparamètres pour `CrossValidator` à examiner.

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .build())

Chaque fois que vous appelez `CrossValidator` dans MLlib, Databricks suit automatiquement toutes les exécutions en utilisant [MLflow] (https://mlflow.org/). Vous pouvez utiliser l'interface utilisateur de MLflow 
([AWS](https://docs.databricks.com/applications/mlflow/index.html) | [Azure](https://docs.microsoft.com/azure/databricks/applications/ mlflow /)) pour comparer les performances de chaque modèle.

Dans cet exemple, nous utilisons le pipeline que nous avons créé comme estimateur

In [0]:
# Create a 3-fold CrossValidator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=bcEvaluator, numFolds=3, parallelism = 4)

# Run cross validations. This step takes a few minutes and returns the best model found from the cross validation.
cvModel = cv.fit(trainDF)

/databricks/spark/python/pyspark/ml/util.py:762: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

## Étape 6. Faire des prédictions et évaluer les performances du modèle
Utilisez le meilleur modèle identifié par la validation croisée pour faire des prédictions sur l'ensemble de données de test, puis évaluez les performances du modèle en utilisant la zone sous la courbe ROC.

In [0]:
# Use the model identified by the cross-validation to make predictions on the test dataset
cvPredDF = cvModel.transform(testDF)

# Evaluate the model's performance based on area under the ROC curve and accuracy 
print(f"Area under ROC curve: {bcEvaluator.evaluate(cvPredDF)}")
print(f"Accuracy: {mcEvaluator.evaluate(cvPredDF)}")

## Étape 7. Autres classificateurs ( À vous )
Tester les classificateurs suivantes: SVM, Radom Forest. Quel classificateur donne de meilleurs resultat en term de courve ROC et precision ? Répeter le pipeline avec ces classificateurs?
Est-ce qu'on a besoin de utiliser tous les variables, trouver les variables plus rélevants, et déveloper un model avec moins de variables? 

**Rendre un document pdf avec des captures de écran et explications des meilleurs modéles et le Notebook**

[SVM](https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-support-vector-machine)
[Radom Forest](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier)